In [1]:
%mkdir -p emperical_eq_rare_tp_f1

In [2]:
import pandas as pd

In [3]:
from scipy.stats import mannwhitneyu, median_test
from statsmodels.stats.multitest import multipletests

from tqdm.auto import tqdm
import numpy as np

/home/paul/miniconda3/envs/skbio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import statsmodels

statsmodels.__version__

'0.14.4'

In [4]:
from scipy.stats import norm

def hodges_lehmann(vals, alpha=0.01):

    A, B = vals

    n = len(A)
    m = len(B)

    M = list(sorted([a - b for a in A for b in B]))

    # https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test#Normal_approximation_and_tie_correction
    # half of sample size +- z score for CI * pooled std of sample size
    # expected U H0, sd U H0
    ZN = norm.ppf(1 - alpha / 2)  # one of tails
    EUH0 = n * m / 2
    SUH0 = (n * m * (n + m + 1) / 12)**0.5

    L = EUH0 - ZN * SUH0
    U = EUH0 + ZN * SUH0

    # for python
    lower = int(round(L) - 1)
    upper = int(round(U) - 1)

    # for boundaries
    lower = max(lower, 0)
    upper = min(upper, len(M) - 1)

    lower = M[lower]
    upper = M[upper]

    return (lower, upper)

In [5]:
def one_sample_hodges_lehmann(val):
    
    M = list(sorted([(a + b) / 2 for a in val for b in val]))

    return np.median(M)

In [6]:
def pseuodo_log(x, pseudo_count=1):
    x = np.array(x) + pseudo_count
    x = np.log(x)
    return x

In [7]:
def get_f1(YY, beta=1):

    # Use the F2 score when recall is more important than precision.
    # F0.5 score: Use the F0.5 score when precision is more important than recall.

    beta = beta**2

    GTP = RES.marker.sum()
    TP = RES.astype('float')[RES.marker].sum().loc[YY]
    FP = RES[~RES.marker.astype('bool')].astype('float').sum().loc[YY]

    PRECISION = TP / (TP + FP) if TP > 0 else 0
    RECALL = TP / GTP

    F1 = (1 + beta) * (PRECISION * RECALL) / (beta * PRECISION + RECALL) if ((PRECISION != 0) and (RECALL != 0)) else 0

    return round(F1 * 100, 1)

In [8]:
DATASETSIZE = 1000 # number of features
MARKERSMOD = 100 # -> 10 markers per 1000

In [9]:
SEED = 0

RESS = {}

# for shift
for MULTIPLIER in [0.15, 0.25, 0.5]:

    # for sample size
    for SIZE in [15, 25, 50, 100]:
        
        # number of trials, 100 trials * 1000 features = 100k tests
        t = tqdm(range(100))
        VALS = {}

        for ATT in t:

            RES = {}

            for TEST in range(DATASETSIZE):

                ISMARKER = TEST % MARKERSMOD == 0
                SEED += 1
                np.random.seed(SEED)

                DRAW = True

                while DRAW:
                    S = np.random.negative_binomial(np.random.randint(1, 10),
                                                    np.random.uniform(),
                                                    size=SIZE * 2)
                    S = S.astype('float').tolist()

                    if len(set(S)) > 1:
                        DRAW = False

                S1 = S[:SIZE]
                S2 = S[SIZE:]

                if ISMARKER:
                    # shift by given margin
                    S2 = S2 + one_sample_hodges_lehmann(S) * MULTIPLIER

                U, p1 = mannwhitneyu(S1, S2)
                try:
                    p2 = median_test(S1, S2).pvalue
                except:
                    p2 = 1

                ci = CI = hodges_lehmann([S1, S2], 0.05)
                strict05 = np.sign(ci[0]) == np.sign(
                    ci[1]) and (ci[0] != 0 and ci[1] != 0)

                ci = CI = hodges_lehmann([S1, S2], 0.01)
                strict01 = np.sign(ci[0]) == np.sign(
                    ci[1]) and (ci[0] != 0 and ci[1] != 0)

                ci = CI = hodges_lehmann([S1, S2], 0.001)
                strict001 = np.sign(ci[0]) == np.sign(
                    ci[1]) and (ci[0] != 0 and ci[1] != 0)

                d = (2 * U) / (SIZE**2) - 1
                d = abs(d)

                RES[TEST] = {
                    'MWU': p1,
                    'MMT': p2,
                    'HL-NO_0.05': strict05,
                    'HL-NO_0.01': strict01,
                    'HL-NO_0.001': strict001,
                    'd': d,
                    'marker': ISMARKER
                }

            RES = pd.DataFrame(RES).T

            RES['MWU-FDRBH'] = multipletests(RES['MWU'], method='fdr_bh')[1]
            RES['MMT-FDRBH'] = multipletests(RES['MMT'], method='fdr_bh')[1]

            for r in ['MWU', 'MMT', 'MWU-FDRBH', 'MMT-FDRBH']:
                RES[f'{r}_0.05'] = RES[r] <= 0.05
                RES[f'{r}_0.01'] = RES[r] <= 0.01
                RES[f'{r}_0.001'] = RES[r] <= 0.001

            RES['CD_0.15'] = RES['d'] >= 0.15
            RES['CD_0.33'] = RES['d'] >= 0.33
            RES['CD_0.47'] = RES['d'] >= 0.47

            RES['CD_0.15_0.05'] = RES['CD_0.15'] & RES['MWU_0.05']
            RES['CD_0.33_0.05'] = RES['CD_0.33'] & RES['MWU_0.05']
            RES['CD_0.47_0.05'] = RES['CD_0.47'] & RES['MWU_0.05']

            F1 = get_f1('HL-NO_0.05')
            F2 = get_f1('HL-NO_0.01')
            F3 = get_f1('HL-NO_0.001')

            F4 = get_f1('MWU_0.05')
            F5 = get_f1('MWU_0.01')
            F6 = get_f1('MWU_0.001')

            F7 = get_f1('MWU-FDRBH_0.05')
            F8 = get_f1('MWU-FDRBH_0.01')
            F9 = get_f1('MWU-FDRBH_0.001')

            F10 = get_f1('CD_0.15_0.05')
            F11 = get_f1('CD_0.33_0.05')
            F12 = get_f1('CD_0.47_0.05')
            
            F13 = get_f1('MMT_0.05')
            F14 = get_f1('MMT_0.01')
            F15 = get_f1('MMT_0.001')

            F16 = get_f1('MMT-FDRBH_0.05')
            F17 = get_f1('MMT-FDRBH_0.01')
            F18 = get_f1('MMT-FDRBH_0.001')

            RESS[(ATT, MULTIPLIER, SIZE)] = {
                'HL-NO_0.05': F1,
                'HL-NO_0.01': F2,
                'HL-NO_0.001': F3,
                'MWU_0.05': F4,
                'MWU_0.01': F5,
                'MWU_0.001': F6,
                'MWU-FDRBH_0.05': F7,
                'MWU-FDRBH_0.01': F8,
                'MWU-FDRBH_0.001': F9,
                
                
                'MMT_0.05': F13,
                'MMT_0.01': F14,
                'MMT_0.001': F15,
                'MMT-FDRBH_0.05': F16,
                'MMT-FDRBH_0.01': F17,
                'MMT-FDRBH_0.001': F18,
                
                'CD_0.15_0.05': F10,
                'CD_0.33_0.05': F11,
                'CD_0.47_0.05': F12,
            }

100%|███████████████████████████████████████████████████████████████████████| 100/100 [05:42<00:00,  3.42s/it]


In [10]:
def f_median(x, precision=2):
    q25 = round(np.nanquantile(x, 0.25), precision)
    q50 = round(np.nanquantile(x, 0.5), precision)
    q75 = round(np.nanquantile(x, 0.75), precision)
    r = f"{q50} (IQR: {q25}; {q75})"
    return r

In [11]:
RESF = pd.DataFrame(RESS).T.reset_index().drop('level_0', axis=1)
RESF.groupby(['level_1', 'level_2']).agg(f_median).reset_index().T.to_excel('emperical_eq_rare_tp_f1/emperical_eq_rare_tp_f1.xlsx')

In [12]:
RESF.to_excel('emperical_eq_rare_tp_f1/emperical_eq_rare_tp_f1-raw.xlsx')

In [13]:
RESF.groupby(['level_1', 'level_2']).median().round(1).T.to_excel('emperical_eq_rare_tp_f1/emperical_eq_rare_tp_f1-simple.xlsx')
RESF.groupby(['level_1', 'level_2']).median().round(1).T

level_1          0.15                    0.25                    0.50        \
level_2           15    25    50    100   15    25    50    100   15    25    
HL-NO_0.05       11.8  17.8  30.8  45.5  15.4  24.0  40.0  56.7  31.2  42.9   
HL-NO_0.01        0.0  15.4  30.8  50.0  13.3  19.6  44.4  70.6  40.0  53.9   
HL-NO_0.001       0.0   0.0  18.2  33.3   0.0   0.0  33.3  59.8  18.2  44.6   
MWU_0.05          6.2   7.9  12.1  17.4   7.2  11.5  17.0  22.9  17.0  21.9   
MWU_0.01          0.0  10.5  20.7  32.6   9.8  16.4  31.6  48.2  31.6  43.2   
MWU_0.001         0.0   0.0  16.7  30.8   0.0   0.0  30.8  57.1  18.2  42.9   
MWU-FDRBH_0.05    0.0   0.0   0.0  18.2   0.0   0.0  18.2  46.2   0.0  18.2   
MWU-FDRBH_0.01    0.0   0.0   0.0  18.2   0.0   0.0   0.0  46.2   0.0  17.4   
MWU-FDRBH_0.001   0.0   0.0   0.0   0.0   0.0   0.0   0.0  18.2   0.0   0.0   
MMT_0.05          5.4   5.7   9.3  14.0   6.5   8.4  13.8  19.6  18.2  20.0   
MMT_0.01          0.0   0.0  12.5  22.2   0.0   0.0  21.1  37.5  15.4  31.2   
MMT_0.001         0.0   0.0   0.0  18.2   0.0   0.0  16.7  33.3   0.0  18.2   
MMT-FDRBH_0.05    0.0   0.0   0.0   0.0   0.0   0.0   0.0  18.2   0.0   0.0   
MMT-FDRBH_0.01    0.0   0.0   0.0   0.0   0.0   0.0   0.0   9.1   0.0   0.0   
MMT-FDRBH_0.001   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
CD_0.15_0.05      6.2   7.9  12.5  18.6   7.2  11.5  17.4  24.6  17.0  21.9   
CD_0.33_0.05      6.2   9.3  19.6  18.2   7.4  13.4  35.3  46.2  17.2  26.1   
CD_0.47_0.05      6.7  12.5   0.0   0.0   8.2  15.4  18.2   0.0  25.7  48.8   

level_1                      
level_2           50    100  
HL-NO_0.05       53.6  64.0  
HL-NO_0.01       75.0  84.2  
HL-NO_0.001      72.8  87.3  
MWU_0.05         25.2  26.0  
MWU_0.01         57.6  59.6  
MWU_0.001        73.7  84.2  
MWU-FDRBH_0.05   66.7  84.2  
MWU-FDRBH_0.01   57.1  82.4  
MWU-FDRBH_0.001  46.2  82.4  
MMT_0.05         26.4  26.7  
MMT_0.01         50.0  58.3  
MMT_0.001        51.6  70.6  
MMT-FDRBH_0.05   46.2  70.6  
MMT-FDRBH_0.01   33.3  66.7  
MMT-FDRBH_0.001  18.2  57.1  
CD_0.15_0.05     25.6  28.4  
CD_0.33_0.05     70.0  82.4  
CD_0.47_0.05     57.1  57.1

In [14]:
RESF.groupby(['level_1', 'level_2']).median().round(1).T.idxmax(axis=0)

level_1  level_2
0.15     15          HL-NO_0.05
         25          HL-NO_0.05
         50          HL-NO_0.05
         100         HL-NO_0.01
0.25     15          HL-NO_0.05
         25          HL-NO_0.05
         50          HL-NO_0.01
         100         HL-NO_0.01
0.50     15          HL-NO_0.01
         25          HL-NO_0.01
         50          HL-NO_0.01
         100        HL-NO_0.001
dtype: object

In [28]:
SUB = RESF.groupby(['level_1', 'level_2']).agg(f_median).reset_index()

ORDER = [
    'MWU_0.05', 'MWU-FDRBH_0.05', 'MMT_0.05', 'MMT-FDRBH_0.05', 'HL-NO_0.05'
]

ORDER += [
    'MWU_0.01', 'MWU-FDRBH_0.01', 'MMT_0.01', 'MMT-FDRBH_0.01', 'HL-NO_0.01'
]

ORDER += [
    'MWU_0.001', 'MWU-FDRBH_0.001', 'MMT_0.001', 'MMT-FDRBH_0.001', 'HL-NO_0.001'
]

ORDER += ['CD_0.15_0.05', 'CD_0.33_0.05', 'CD_0.47_0.05']

SUB[SUB.level_1==0.5].T.loc[ORDER]

,8,9,10,11
MWU_0.05,17.0 (IQR: 13.95; 20.15),21.9 (IQR: 18.48; 24.3),25.2 (IQR: 22.42; 28.6),26.0 (IQR: 23.65; 28.2)
MWU-FDRBH_0.05,0.0 (IQR: 0.0; 0.0),18.2 (IQR: 17.82; 33.3),66.7 (IQR: 57.1; 75.0),84.2 (IQR: 77.8; 90.0)
MMT_0.05,18.2 (IQR: 12.4; 24.25),20.0 (IQR: 15.8; 25.0),26.4 (IQR: 21.7; 31.02),26.7 (IQR: 23.0; 29.6)
MMT-FDRBH_0.05,0.0 (IQR: 0.0; 0.0),0.0 (IQR: 0.0; 18.2),46.2 (IQR: 33.3; 57.1),70.6 (IQR: 57.1; 75.0)
HL-NO_0.05,31.2 (IQR: 25.8; 38.7),42.9 (IQR: 36.22; 48.3),53.55 (IQR: 49.7; 61.5),64.0 (IQR: 57.1; 69.2)
MWU_0.01,31.6 (IQR: 23.5; 40.42),43.2 (IQR: 32.0; 50.48),57.6 (IQR: 50.0; 64.0),59.65 (IQR: 53.3; 66.7)
MWU-FDRBH_0.01,0.0 (IQR: 0.0; 0.0),17.45 (IQR: 0.0; 18.2),57.1 (IQR: 46.2; 66.7),82.4 (IQR: 75.0; 88.9)
MMT_0.01,15.4 (IQR: 0.0; 27.18),31.2 (IQR: 18.8; 40.0),50.0 (IQR: 40.0; 60.9),58.3 (IQR: 50.0; 64.08)
MMT-FDRBH_0.01,0.0 (IQR: 0.0; 0.0),0.0 (IQR: 0.0; 0.0),33.3 (IQR: 18.2; 46.2),66.7 (IQR: 57.1; 75.0)
HL-NO_0.01,40.0 (IQR: 30.8; 50.0),53.9 (IQR: 42.9; 66.7),75.0 (IQR: 66.7; 80.0),84.2 (IQR: 77.8; 90.0)


In [31]:
70.6 - 46.2

24.39999999999999